In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
import pycolmap
from pathlib import Path

import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

from tqdm import tqdm

from hloc.utils.io import list_h5_names, get_matches

from imc2023.utils.eval import eval

import os
import pycolmap
from pathlib import Path
import numpy as np
from collections import defaultdict
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from hloc.utils.io import list_h5_names, get_matches, get_keypoints
from hloc.visualization import plot_images, plot_keypoints, plot_matches, read_image, add_text, cm_RdGn

In [ ]:
DIR = Path("../image-matching-challenge-2023")
OUT = Path("../outputs")
MODE = "train"
dataset = "heritage" # "heritage", "haiper", "urban"
scene = "cyprus" # "dioscuri", "cyprus", "wall", "kyiv-puppet-theater", "bike", "chairs", "fountain"

# SG vs LG

In [ ]:
scores = {
    "SP+LG": {
        "train": 0.650,
        "public": np.mean([.388, .382, .382]),
        "private": np.mean([0.471, .454, .457]),
    },
    "SP+SG": {
        "train": 0.643,
        "public": np.mean([.356, .356, .370]),
        "private": np.mean([0.443, .435, .437]),
    },
}

fig, ax = plt.subplots(1, 1, figsize=(10, 7))

for config in ["SP+SG", "SP+LG"]:
    ax.bar(
        np.array([0, 1, 2]) + 0.2 * (1 if config == "SP+LG" else -1),
        [
            scores[config]["train"],
            scores[config]["public"],
            scores[config]["private"],
        ],
        label=config,
        width=0.35,
        alpha=0.3
    )

ax.set_xticks([0, 1, 2])
ax.set_xticklabels(["Train", "Public", "Private"])
ax.set_ylabel("mAA", fontsize=18)
ax.legend(fontsize=18)

ax.set_ylim([0.3, 0.8])

# make font size 18
ax.tick_params(axis="both", which="major", labelsize=18)

plt.tight_layout()
plt.show()

print("Train: ", scores["SP+LG"]["train"], scores["SP+SG"]["train"])
print("Public: ", scores["SP+LG"]["public"], scores["SP+SG"]["public"])
print("Private: ", scores["SP+LG"]["private"], scores["SP+SG"]["private"])


In [ ]:
scores = {
    "w/o hloc": {
        "train": 0.83,
        "public": 0.462,
        "private": 0.505,
    },
    "hloc": {
        "train": 0.837,
        "public": 0.475,
        "private": 0.523,
    },
}

fig, ax = plt.subplots(1, 1, figsize=(10, 4))

for config in ["w/o hloc", "hloc"]:
    ax.bar(
        np.array([0, 1, 2]) + 0.2 * (1 if config == "hloc" else -1),
        [
            scores[config]["train"],
            scores[config]["public"],
            scores[config]["private"],
        ],
        label=config,
        width=0.35,
        alpha=0.3
    )

ax.set_xticks([0, 1, 2])
ax.set_xticklabels(["Train", "Public", "Private"])
ax.set_ylabel("mAA", fontsize=18)
ax.legend(fontsize=18)

ax.set_ylim([0.4, 1])

# make font size 18
ax.tick_params(axis="both", which="major", labelsize=18)

plt.tight_layout()
plt.show()


In [ ]:
scores = {
    "DISK": {
        "train": 0.761,
        "public": 0.386,
        "private": 0.437,
    },
    "ALIKED2K+DISK+SIFT": {
        "train": 0.837,
        "public": 0.475,
        "private": 0.523,
    },
    "DISK+SP": {
        "train": 0.876,
        "public": 0.484,
        "private": 0.562,
    },
    "DISK+SIFT+SP": {
        "train": 0.890,
        "public": 0.511,
        "private": 0.559,
    },
}

colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]

fig, ax = plt.subplots(1, 1, figsize=(10, 7))

for i, config in enumerate(["ALIKED2K+DISK+SIFT", "DISK"]):
    idx = list(scores.keys()).index(config)
    ax.bar(
        np.array([0, 1, 2]) + 0.2 * (i - 0.5) * 2,
        [
            scores[config]["train"],
            scores[config]["public"],
            scores[config]["private"],
        ],
        label=config,
        width=0.35,
        alpha=0.3,
        color=colors[idx]
    )

ax.set_xticks([0, 1, 2])
ax.set_xticklabels(["Train", "Public", "Private"])
ax.set_ylabel("mAA", fontsize=18)
ax.legend(fontsize=18)

ax.set_ylim([0.3, 1])

# make font size 18
ax.tick_params(axis="both", which="major", labelsize=18)

plt.tight_layout()
plt.show()

In [ ]:
scores = {
    "DISK": {
        "train": 0.761,
        "public": 0.386,
        "private": 0.437,
    },
    "ALIKED2K+DISK+SIFT": {
        "train": 0.837,
        "public": 0.475,
        "private": 0.523,
    },
    "DISK+SP": {
        "train": 0.876,
        "public": 0.484,
        "private": 0.562,
    },
    "DISK+SIFT+SP": {
        "train": 0.890,
        "public": 0.511,
        "private": 0.559,
    },
}

colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]

fig, ax = plt.subplots(1, 1, figsize=(10, 7))

for i, config in enumerate(["ALIKED2K+DISK+SIFT", "DISK+SIFT+SP"]):
    idx = list(scores.keys()).index(config)
    ax.bar(
        np.array([0, 1, 2]) + 0.2 * (i - 0.5) * 2,
        [
            scores[config]["train"],
            scores[config]["public"],
            scores[config]["private"],
        ],
        label=config,
        width=0.35,
        alpha=0.3,
        color=colors[idx]
    )

ax.set_xticks([0, 1, 2])
ax.set_xticklabels(["Train", "Public", "Private"])
ax.set_ylabel("mAA", fontsize=18)
ax.legend(fontsize=18)

ax.set_ylim([0.3, 1])

# make font size 18
ax.tick_params(axis="both", which="major", labelsize=18)

plt.tight_layout()
plt.show()

In [ ]:
scores = {
    "DISK": {
        "train": 0.761,
        "public": 0.386,
        "private": 0.437,
    },
    "ALIKED2K+DISK+SIFT": {
        "train": 0.837,
        "public": 0.475,
        "private": 0.523,
    },
    "DISK+SIFT+SP+SG": {
        "train": 0.867,
        "public": 0,
        "private": 0,
    },
    "DISK+SIFT+SP": {
        "train": 0.890,
        "public": 0.511,
        "private": 0.559,
    },
}

colors = ["#1f77b4", "#ff7f0e", "black", "#d62728"]

fig, ax = plt.subplots(1, 1, figsize=(10, 7))

for i, config in enumerate(["DISK+SIFT+SP+SG", "DISK+SIFT+SP"]):
    idx = list(scores.keys()).index(config)
    name = "DISK+SIFT+SP+LightGlue" if config == "DISK+SIFT+SP" else "DISK+SIFT+SP+SuperGlue"
    ax.bar(
        np.array([0, 1, 2]) + 0.2 * (i - 0.5) * 2,
        [
            scores[config]["train"],
            scores[config]["public"],
            scores[config]["private"],
        ],
        label=name,
        width=0.35,
        alpha=0.3,
        color=colors[idx]
    )

ax.set_xticks([0, 1, 2])
ax.set_xticklabels(["Train", "Public", "Private"])
ax.set_ylabel("mAA", fontsize=18)
ax.legend(fontsize=18)

ax.set_ylim([0.3, 1])

# make font size 18
ax.tick_params(axis="both", which="major", labelsize=18)

plt.tight_layout()
plt.show()

# Runtime

In [ ]:
data = {
    "SP+LG": {
        'preprocessing': 1.282 * 60,
        'pairs-extraction': 0.0003 * 60,
        'feature-extraction': 0.941 * 60,
        'feature-matching': 24.646 * 60, 
        'rotate-keypoints': 0,
        'create-ensemble': 0,
        'sfm': 14.408 * 60,
        'localize-unreg': 0,
    },
    "SP+SG": {
        'preprocessing': 1.282 * 60,
        'pairs-extraction': 0.0003 * 60,
        'feature-extraction': 0.941 * 60,
        'feature-matching': 43.915 * 60,
        'rotate-keypoints': 0,
        'create-ensemble': 0,
        'sfm': 12.178 * 60,
        'localize-unreg': 0,
    },
}

# create overall runtime plot
fig, ax = plt.subplots(1, 1, figsize=(10, 7))

legend_steps = ['preprocessing', 'pairs-extraction', 'feature-extraction', 'feature-matching', 'create-ensemble', 'sfm', 'localize-unreg']

cmap = plt.get_cmap("tab20")

for r in data: #['SP-rot-sci', 'SP-rot-pixsfm-sci', 'SP+SG-rot-sci', 'SP+SG-rot-pixsfm-sci', ]:
    name = r
    mean_times = [v for k, v in data[r].items()]
    pipline_steps = [k for k, v in data[r].items()]
    
    # mean_times = [t / len(df) for t in mean_times]

    cumsum = 0
    for idx, step in enumerate(pipline_steps):
        ax.bar(
            name if name != "SP" else "SP+LG",
            mean_times[idx],
            bottom=cumsum,
            label=step if r == "SP+LG" and step in legend_steps else "",
            color=cmap(idx),
        )
        cumsum += mean_times[idx]

        if step == "feature-matching":
            minutes = np.floor(mean_times[idx] / 60)
            sec = mean_times[idx] % 60
            print(f"{name}: {minutes:.0f} min {sec:.0f} sec")

    # rotate the xticklabels
    # for tick in ax.get_xticklabels():
        # tick.set_rotation(20)
        # tick.set_ha("right")

    # divide by 60 to get minutes
    def divide_by_60(x, pos):
        return f'{x/60:.0f}'

    # Create a formatter using the custom function
    formatter = ticker.FuncFormatter(divide_by_60)

    # Apply the formatter to the x-axis
    ax.yaxis.set_major_formatter(formatter)  

    # increase fonts
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.tick_params(axis='both', which='minor', labelsize=18)

    ax.set_ylabel("Runtime (min)", fontsize=18)

ax.set_ylim([0, 66*60])

plt.tight_layout()
plt.legend(fontsize=18)
plt.savefig("/Users/alexanderveicht/Desktop/runtime.pdf")
plt.show()

# Helper Functions

In [ ]:
"""Tool to visualize epipolar lines in a pair of images.

2D visualization primitives based on Matplotlib.
1) Plot images with `plot_images`.
2) Call `plot_keypoints` or `plot_matches` any number of times.
3) Optionally: save a .png or .pdf plot (nice in papers!) with `save_plot`.
"""

from typing import Optional, Union

import matplotlib
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import numpy as np
import pycolmap


def cm_RdGn(x: np.ndarray) -> np.ndarray:
    """Custom colormap: red (0) -> yellow (0.5) -> green (1).

    Args:
        x (np.ndarray): Array of values in [0, 1].

    Returns:
        np.ndarray: Array of RGB values in [0, 1].
    """
    x = np.clip(x, 0, 1)[..., None] * 2
    c = x * np.array([[0, 1.0, 0]]) + (2 - x) * np.array([[1.0, 0, 0]])
    return np.clip(c, 0, 1)


def plot_images(
    imgs: list,
    titles: Optional[list] = None,
    cmaps: Union[str, list, tuple] = "gray",
    dpi: int = 100,
    pad: float = 0.5,
    adaptive: bool = True,
) -> None:
    """Plot a set of images horizontally.

    Args:
        imgs (list): A list of NumPy or PyTorch images, RGB (H, W, 3) or mono (H, W).
        titles (list, optional): A list of strings, as titles for each image. Defaults to None.
        cmaps (str, optional): Colormaps for monochrome images. Defaults to "gray".
        dpi (int, optional): Dpi of the figure. Defaults to 100.
        pad (float, optional): Padding of the figure. Defaults to 0.5.
        adaptive (bool, optional): Whether the figure size should fit the image aspect ratios.
        Defaults to True.
    """
    n = len(imgs)
    if not isinstance(cmaps, (list, tuple)):
        cmaps = [cmaps] * n

    ratios = [i.shape[1] / i.shape[0] for i in imgs] if adaptive else [4 / 3] * n

    figsize = [sum(ratios) * 4.5, 4.5]

    fig, ax = plt.subplots(1, n, figsize=figsize, dpi=dpi, gridspec_kw={"width_ratios": ratios})
    if n == 1:
        ax = [ax]

    for i in range(n):
        ax[i].imshow(imgs[i], cmap=plt.get_cmap(cmaps[i]))
        ax[i].get_yaxis().set_ticks([])
        ax[i].get_xaxis().set_ticks([])
        ax[i].set_axis_off()
        for spine in ax[i].spines.values():  # remove frame
            spine.set_visible(False)
        if titles:
            ax[i].set_title(titles[i])

    fig.tight_layout(pad=pad)


def plot_keypoints(
    kpts: list,
    colors: Union[str, list] = "lime",
    ps: float = 4,
    axes: Optional[plt.axes] = None,
    a: float = 1.0,
) -> None:
    """Plot keypoints for existing images.

    Args:
        kpts (list): List of ndarrays of size (N, 2).
        colors (Union[str, list], optional): String or list of list of tuples (one for each kpt).
        ps (float, optional): Size of the keypoints as float. Defaults to 4.
        axes (plt.axes, optional): Axes to plot on. Defaults to None.
        a (float, optional): Alpha opacity of the keypoints. Defaults to 1.0.
    """
    """Plot keypoints for existing images.
    Args:
        kpts: list of ndarrays of size (N, 2).
        colors: string, or list of list of tuples (one for each keypoints).
        ps: size of the keypoints as float.
    """
    if not isinstance(colors, list):
        colors = [colors] * len(kpts)
    if axes is None:
        axes = plt.gcf().axes
    assert len(axes) == 2
    for ax, k, c in zip(axes, kpts, colors):
        ax.scatter(k[:, 0], k[:, 1], c=c, s=ps, linewidths=0, alpha=a)


def plot_matches(
    kpts0: np.ndarray,
    kpts1: np.ndarray,
    color: Optional[list] = None,
    lw: float = 1.5,
    ps: float = 4,
    a: float = 1.0,
    labels: Optional[list] = None,
    axes: Optional[plt.axes] = None,
):
    """Plot matches for a pair of existing images.

    Args:
        kpts0 (np.ndarray): List of ndarrays of size (N, 2).
        kpts1 (np.ndarray): Corresponding keypoints of size (N, 2).
        color (list, optional): Color of each match, string or RGB tuple. Random if not given.
        lw (float, optional): Width of the lines. Defaults to 1.5.
        ps (float, optional): Size of the end points (no endpoint if ps=0). Defaults to 4.
        a (float, optional): Alpha opacity of the match lines. Defaults to 1.0.
        labels (list, optional): Labels for each match. Defaults to None.
        axes (plt.axes, optional): Axes to plot on. Defaults to None.
    """
    fig = plt.gcf()
    if axes is None:
        ax = fig.axes
        ax0, ax1 = ax[0], ax[1]
    else:
        ax0, ax1 = axes
    fig.canvas.draw()

    assert len(kpts0) == len(kpts1)
    if color is None:
        color = matplotlib.cm.hsv(np.random.rand(len(kpts0))).tolist()
    elif len(color) > 0 and not isinstance(color[0], (tuple, list)):
        color = [color] * len(kpts0)

    if lw > 0:
        for i in range(len(kpts0)):
            line = matplotlib.patches.ConnectionPatch(
                xyA=(kpts0[i, 0], kpts0[i, 1]),
                xyB=(kpts1[i, 0], kpts1[i, 1]),
                coordsA=ax0.transData,
                coordsB=ax1.transData,
                axesA=ax0,
                axesB=ax1,
                zorder=1,
                color=color[i],
                linewidth=lw,
                clip_on=True,
                alpha=a,
                label=None if labels is None else labels[i],
                picker=5.0,
            )
            line.set_annotation_clip(True)
            fig.add_artist(line)

    # freeze the axes to prevent the transform to change
    ax0.autoscale(enable=False)
    ax1.autoscale(enable=False)

    if ps > 0:
        ax0.scatter(kpts0[:, 0], kpts0[:, 1], c=color, s=ps)
        ax1.scatter(kpts1[:, 0], kpts1[:, 1], c=color, s=ps)


def add_text(
    idx: int,
    text: str,
    pos: tuple = (0.01, 0.99),
    fs: float = 15,
    color: str = "w",
    lcolor: str = "k",
    lwidth: float = 2,
    ha: str = "left",
    va: str = "top",
) -> None:
    """Add text to an existing image.

    Args:
        idx (int): Index of the image to add text to.
        text (str): Text to add.
        pos (tuple, optional): Position of the text as a tuple (x, y). Defaults to (0.01, 0.99).
        fs (float, optional): Font size. Defaults to 15.
        color (str, optional): Color of the text. Defaults to "w".
        lcolor (str, optional): Color of the text outline. Defaults to "k".
        lwidth (float, optional): Width of the text outline. Defaults to 2.
        ha (str, optional): Horizontal alignment. Defaults to "left".
        va (str, optional): Vertical alignment. Defaults to "top".
    """
    ax = plt.gcf().axes[idx]
    t = ax.text(*pos, text, fontsize=fs, ha=ha, va=va, color=color, transform=ax.transAxes)
    if lcolor is not None:
        t.set_path_effects(
            [path_effects.Stroke(linewidth=lwidth, foreground=lcolor), path_effects.Normal()]
        )


def draw_epipolar_line(
    line: np.ndarray,
    imshape: tuple,
    axis: plt.axis,
    color: str = "b",
    label: Optional[str] = None,
    alpha: float = 1.0,
    visible: bool = True,
) -> None:
    """Draw an epipolar line on an existing image.

    Args:
        line (np.ndarray): Line to draw.
        imshape (tuple): Shape of the image.
        axis (plt.axis): Axis to draw on.
        color (str, optional): Color of the line. Defaults to "b".
        label (str, optional): Label of the line. Defaults to None.
        alpha (float, optional): Alpha opacity of the line. Defaults to 1.0.
        visible (bool, optional): Whether the line is visible. Defaults to True.
    """
    h, w = imshape[:2]
    # Intersect line with lines representing image borders.
    X1 = np.cross(line, [1, 0, -1])
    X1 = X1[:2] / X1[2]
    X2 = np.cross(line, [1, 0, -w])
    X2 = X2[:2] / X2[2]
    X3 = np.cross(line, [0, 1, -1])
    X3 = X3[:2] / X3[2]
    X4 = np.cross(line, [0, 1, -h])
    X4 = X4[:2] / X4[2]

    # Find intersections which are not outside the image,
    # which will therefore be on the image border.
    Xs = [X1, X2, X3, X4]
    Ps = []
    for p in range(4):
        X = Xs[p]
        if (0 <= X[0] <= (w + 1e-6)) and (0 <= X[1] <= (h + 1e-6)):
            Ps.append(X)
            if len(Ps) == 2:
                break

    # Plot line, if it's visible in the image.
    if len(Ps) == 2:
        axis.plot(
            [Ps[0][0], Ps[1][0]],
            [Ps[0][1], Ps[1][1]],
            color,
            linestyle="dashed",
            label=label,
            alpha=alpha,
            visible=visible,
        )


def get_line(F: np.ndarray, kp: np.ndarray) -> np.ndarray:
    """Get the epipolar line for a given keypoint.

    Args:
        F (np.ndarray): Fundamental matrix.
        kp (np.ndarray): Keypoint.

    Returns:
        np.ndarray: Epipolar line for the given keypoint.
    """
    hom_kp = np.array([list(kp) + [1.0]]).transpose()
    return np.dot(F, hom_kp)


def plot_epipolar_lines(
    pts0: np.ndarray,
    pts1: np.ndarray,
    F: np.ndarray,
    color: str = "b",
    axes: Optional[plt.axes] = None,
    labels: Optional[list] = None,
    a: float = 1.0,
    visible: bool = True,
) -> None:
    """Plot epipolar lines on a pair of images.

    Args:
        pts0 (np.ndarray): Points in the first image.
        pts1 (np.ndarray): Corresponding points in the second image.
        F (np.ndarray): Fundamental matrix.
        color (str, optional): Color of the lines. Defaults to "b".
        axes (plt.axes, optional): Axes to draw on. Defaults to None.
        labels (list, optional): Labels of the lines. Defaults to None.
        a (float, optional): Alpha opacity of the lines. Defaults to 1.0.
        visible (bool, optional): Whether the lines are visible. Defaults to True.
    """
    if axes is None:
        axes = plt.gcf().axes
    assert len(axes) == 2

    for ax, kps in zip(axes, [pts1, pts0]):
        _, w = ax.get_xlim()
        h, _ = ax.get_ylim()

        imshape = (h + 0.5, w + 0.5)
        for i in range(kps.shape[0]):
            if ax == axes[0]:
                line = get_line(F.T, kps[i])[:, 0]
            else:
                line = get_line(F, kps[i])[:, 0]
            draw_epipolar_line(
                line,
                imshape,
                ax,
                color=color,
                label=None if labels is None else labels[i],
                alpha=a,
                visible=visible,
            )

def get_rel_pose(name0, name1, features_path, matches_path, image_dir):
    kp0, kp1 = get_keypoints(features_path, name0), get_keypoints(features_path, name1)
    m, _ = get_matches(matches_path, name0, name1)

    camera0 = pycolmap.infer_camera_from_image(image_dir / name0)
    camera1 = pycolmap.infer_camera_from_image(image_dir / name1)

    options = pycolmap.TwoViewGeometryOptions()
    # options.min_num_inliers = 40
    out = pycolmap.two_view_geometry_estimation(
        kp0[m[:,0]],
        kp1[m[:,1]],
        camera0,
        camera1,
        options,
    )
    if not out["success"]:
        return None, None, 0

    # print(f"Num inlier matches: {np.sum(out['inliers'])} / {len(m)}")

    return out["qvec"], out["tvec"], out["inliers"]


def skew_symmetric_matrix(tvec):
    tx, ty, tz = tvec
    skew_matrix = np.array([[0, -tz, ty],
                            [tz, 0, -tx],
                            [-ty, tx, 0]])
    return skew_matrix

def get_fundamental_matrix(R, tvec, K0, K1):
    # K′−T[t]×RK−1

    cross_t = skew_symmetric_matrix(tvec)
    return np.linalg.inv(K1).T @ cross_t @ R @ np.linalg.inv(K0)
    
def get_F_from_model(name0, name1, model):
    img0 = model.find_image_with_name(name0)
    img1 = model.find_image_with_name(name1)

    qvec0, tvec0 = np.array(img0.qvec), np.array(img0.tvec)
    qvec1, tvec1 = np.array(img1.qvec), np.array(img1.tvec)

    qvec, tvec = pycolmap.relative_pose(qvec0, tvec0, qvec1, tvec1)

    R = pycolmap.qvec_to_rotmat(qvec)
    
    img0 = model.find_image_with_name(name0)
    img1 = model.find_image_with_name(name1)

    camera0 = model.cameras[img0.camera_id]
    camera1 = model.cameras[img1.camera_id]

    K0 = np.array(camera0.calibration_matrix())
    K1 = np.array(camera1.calibration_matrix())


    return get_fundamental_matrix(R, tvec, K0, K1)

# Rotations

In [ ]:
NAME = "DISK-rot-pixsfm-sci"

img_dir = OUT / NAME / dataset / scene / "images"
rot_img_dir = OUT / NAME / dataset / scene / "images_rotated"

In [ ]:
image_names = sorted(os.listdir(img_dir))

for img_name in image_names[:10]:
    plot_images(
        [read_image(img_dir / img_name), read_image(rot_img_dir / img_name)],
        titles=["Original", "Rotated"],
    )
    plt.show()

In [ ]:
keypoints = OUT / NAME / dataset / scene / "feats-disk.h5"

for img_name in image_names[:10]:
    plot_images(
        [read_image(img_dir / img_name), read_image(rot_img_dir / img_name)],
        titles=["Original", "Rotated"],
    )
    plot_keypoints([get_keypoints(keypoints, img_name), get_keypoints(keypoints, img_name)])
    plt.show()

# Pairs

In [ ]:
NAME = "DISK-rot-pixsfm-sci"

scene_dir = Path(f"../outputs/{NAME}/{dataset}/{scene}")
images = Path(f"{DIR}/{MODE}/{dataset}/{scene}/images")

features = scene_dir / "features.h5"
matches = scene_dir / "matches.h5"

image_names = sorted(list_h5_names(features))
pairs = sorted(list_h5_names(matches))

pairs = np.random.permutation(pairs)

for p in pairs:
    name0, name1 = p.split("/")

    if name0 != "DSC_6492.JPG":
        continue

    print(f"\"{name0}\", \"{name1}\"")
    
    plot_images(
        [read_image(images / name0), read_image(images / name1)],
        [name0, name1]
    )
    plt.show()

# Keypoints & Matches

In [ ]:
name0, name1 = "DSC_6492.JPG", "DSC_6597.JPG"
print(name0, name1)

In [ ]:
def plot_kpts_and_matches():
    scene_dir = Path(f"../outputs/{NAME}/{dataset}/{scene}")
    images = Path(f"{DIR}/{MODE}/{dataset}/{scene}/images")

    features = scene_dir / "features.h5"
    matches = scene_dir / "matches.h5"

    plot_images(
        [read_image(images / name0), read_image(images / name1)],
        [name0, name1]
    )

    plot_keypoints(
        [get_keypoints(features, name0), get_keypoints(features, name1)],
        colors=["lime", "lime"],
    )

    plt.show()

    plot_images(
        [read_image(images / name0), read_image(images / name1)],
        [name0, name1]
    )

    kp0, kp1 = get_keypoints(features, name0), get_keypoints(features, name1)
    
    plot_keypoints(
        [kp0, kp1],
        colors=["lime", "lime"],
    )

    m, _ = get_matches(matches, name0, name1)

    inliers = get_rel_pose(name0, name1, features, matches, images)[-1]
    
    if inliers != 0:
        inliers = np.array(inliers)
        m_in = m[inliers]
        m_out = m[~inliers]

        plot_matches(
            kpts0=kp0[m_out[:, 0]],
            kpts1=kp1[m_out[:, 1]],
            color="r",
            a=0.3,
        )
        plot_matches(
            kpts0=kp0[m_in[:, 0]],
            kpts1=kp1[m_in[:, 1]],
            color="lime",
            a=0.3,
        )
        add_text(0, f"Inliers {sum(inliers)} / {len(m)}", fs=25)
    else:
        plot_matches(
            kpts0=kp0[m[:, 0]],
            kpts1=kp1[m[:, 1]],
            color="r",
            a=0.3,
        )
        add_text(0, f"Inliers 0 / {len(m)}", fs=25)

In [ ]:
NAME = "DISK-rot-pixsfm-sci"
plot_kpts_and_matches()
plt.savefig(f"/Users/alexanderveicht/Desktop/{NAME}-{name0}-{name1}.pdf")

In [ ]:
NAME = "ALIKED-rot-pixsfm-sci"
plot_kpts_and_matches()
plt.savefig(f"/Users/alexanderveicht/Desktop/{NAME}-{name0}-{name1}.pdf")

In [ ]:
NAME = "SIFT-rot-pixsfm-sci"
plot_kpts_and_matches()

plt.savefig(f"/Users/alexanderveicht/Desktop/{NAME}-{name0}-{name1}.pdf")

In [ ]:
NAME = "ALIKED2K+DISK+SIFT-rot-pixsfm-sci"
plot_kpts_and_matches()

plt.savefig(f"/Users/alexanderveicht/Desktop/{NAME}-{name0}-{name1}.pdf")

# Unregistered

In [ ]:
name0, name1 = "DSC_6589.JPG", "DSC_6585.JPG"
NAME = "ALIKED2K+DISK+SIFT-rot-pixsfm-sci"
plot_kpts_and_matches()

# Compare Matches

In [ ]:
gt_model = pycolmap.Reconstruction(DIR / MODE / dataset / scene / "sfm")
print(gt_model.summary())

In [ ]:
def plot_pairs():
    scene_dir = Path(f"../outputs/{NAME}/{dataset}/{scene}")

    images = Path(f"{DIR}/{MODE}/{dataset}/{scene}/images")

    features = scene_dir / "features.h5"
    matches = scene_dir / "matches.h5"

    model = pycolmap.Reconstruction(scene_dir / "sparse")
    print(model.summary())

    image_names = sorted(list_h5_names(features))
    pairs = sorted(list_h5_names(matches))

    for p in pairs[:]:
        name0, name1 = p.split("/")
        print(name0, name1)

        # if (name0, name1) != ("DSC_6480.JPG", "DSC_6605.JPG"):
        #     continue

        qvec, tvec, inliers = get_rel_pose(name0, name1, features, matches, images)

        if qvec is None:
            continue

        R = pycolmap.qvec_to_rotmat(qvec)
        
        img0 = model.find_image_with_name(name0)
        img1 = model.find_image_with_name(name1)

        camera0 = model.cameras[img0.camera_id]
        camera1 = model.cameras[img1.camera_id]

        K0 = np.array(camera0.calibration_matrix())
        K1 = np.array(camera1.calibration_matrix())


        F = get_fundamental_matrix(R, tvec, K0, K1)

        kp0, kp1 = get_keypoints(features, name0), get_keypoints(features, name1)
        m, _ = get_matches(matches, name0, name1)

        plot_images(
            [read_image(images / name0), read_image(images / name1)],
            titles=[name0, name1],
        )
        inliers = np.array(inliers)
        m_in = m[inliers]
        m_out = m[~inliers]
        plot_matches(kp0[m_in[:,0]], kp1[m_in[:,1]], color="lime", a=0.3)
        plot_matches(kp0[m_out[:,0]], kp1[m_out[:,1]], color="r", a=0.3)

        F_gt = get_F_from_model(name0, name1, gt_model)
        plot_epipolar_lines(kp0[m_in[:,0]], kp1[m_in[:,1]], F_gt, color="blue", a=0.3)
        plot_epipolar_lines(kp0[m_in[:,0]], kp1[m_in[:,1]], F, color="lime", a=0.3)

        add_text(0, f"Inliers: {inliers.sum()} / {len(m)}")

        plt.show()

        break

NAME = "DISK-rot-pixsfm-sci"
plot_pairs()

In [ ]:
NAME = "ALIKED+DISK-rot-pixsfm-sci"
plot_pairs()

In [ ]:
NAME = "ALIKED2K+DISK+SIFT-rot-pixsfm-sci"
plot_pairs()